# Finetune ruGPT3Small on essays

## Install enviroment

In [ ]:
!pip3 install urllib3==1.25.4

In [ ]:
!pip3 install transformers==2.8.0

In [ ]:
!wget https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/pretrain_transformers.py

In [ ]:
!wget https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/generate_transformers.py

In [ ]:
%%writefile setup.sh

export CUDA_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

In [ ]:
!sh setup.sh

## Run finetuning
The following code download our model and tokenizer from transformers and finetune model essays.

This took aroung ten minutes and obtain perplexity = tensor(13.8065)

In [ ]:
!python pretrain_transformers.py \
    --output_dir=russian_model \
    --model_type=gpt2 \
    --model_name_or_path=sberbank-ai/rugpt3small_based_on_gpt2 \
    --do_train \
    --train_data_file=russian_dataset.txt \
    --fp16 \
    --per_gpu_train_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --num_train_epochs 5 \
    --save_steps=1500 \
    --block_size=1024 \
    --overwrite_output_dir

## Check our model

In [8]:
!wget https://raw.githubusercontent.com/sberbank-ai/ruGPT3_essays/main/test/russian_test.json
!wget https://raw.githubusercontent.com/sberbank-ai/ruGPT3_essays/main/test/literature_test.json
!wget https://raw.githubusercontent.com/sberbank-ai/ruGPT3_essays/main/test/history_test.json

--2020-11-22 20:08:18--  https://raw.githubusercontent.com/sberbank-ai/ruGPT3_essays/main/test/russian_test.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 393083 (384K) [text/plain]
Saving to: ‘russian_test.json.1’

russian_test.json.1 100%[===================>] 383.87K  --.-KB/s    in 0.03s   

2020-11-22 20:08:18 (14.9 MB/s) - ‘russian_test.json.1’ saved [393083/393083]

--2020-11-22 20:08:18--  https://raw.githubusercontent.com/sberbank-ai/ruGPT3_essays/main/test/literature_test.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 

In [ ]:
!python generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path=russian_model \
    --filename=russian_test.json \
    --k=25 \
    --p=0.8 \
    --temperature=0.2 \
    --repetition_penalty=1.2 \
    --length=700